# Path Tracking Strategies

In [6]:
import numpy as np
import matplotlib.path as path
import import_ipynb
import utils

### Deadlock Function

`def deadlock(robots, parameters, verbose)`

Reset the closer point in order to break deadlock

#### Parameters:
* **robots&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: list**
    * List of instances of Robots
* **parameters : dictionary**
    * Global parameters
* **verbose&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: boolean**
    * Flag for verbose

In [ ]:
def deadlock(robots, parameters, verbose):
    # Check all robots positions
    for agent in robots:
        # Check if the robot is in deadlock situation
        if agent.deadlock:
            # Get the sense of the vertices_bvc array
            sense = utils.get_sense(agent.path[-2], agent.bvc)
            #i = 0
            milestone = -1
            #--------------------------CONTROL POINT--------------------------
            if verbose:
                print('Robot in deadlock:', agent.pos_i)
                print('Vertices to check:', agent.bvc)
                print('Sense:', sense)
            main_distance = np.linalg.norm(agent.closer_point - agent.bvc[0])
            for i in range(len(agent.bvc)):
                temp_distance = np.linalg.norm(agent.closer_point
                                               - agent.bvc[i])
                if verbose:
                    print('Distance from vertex to the closest point:',
                          temp_distance)
                if (temp_distance < parameters["tolerance"]
                    and temp_distance < main_distance):
                    milestone = i
                    main_distance = temp_distance
                j = 0
                for j in range(parameters["previous"]):
                    prev_dist = np.linalg.norm(agent.bvc[i] - agent.path[-2-j])
                    if verbose:
                        print('Distance from the ' + str(j)
                              + 'th previus point:', prev_dist)
                    if (prev_dist < parameters["tolerance"]
                        and prev_dist < main_distance):
                        milestone = i
                        main_distance = prev_dist
            #--------------------------CONTROL POINT--------------------------
            if verbose:
                print('Pointer:', milestone)
            if milestone > -1 and milestone < len(agent.bvc):
                #------------------------CONTROL POINT------------------------
                if verbose:
                    print('Deadlock in vertex')
                if sense == 'counterclockwise':
                    if milestone == 0:
                        aux_vertex = (agent.bvc[-2] - agent.bvc[milestone])
                    else:
                        aux_vertex = (agent.bvc[milestone-1]
                                      - agent.bvc[milestone])
                    aux_vertex = aux_vertex / np.linalg.norm(aux_vertex)
                    position_candidate =(aux_vertex*parameters["deadlock_mov"]
                                         + agent.pos_i)
                elif sense == 'clockwise':
                    aux_vertex = (agent.bvc[milestone+1]- agent.bvc[milestone])
                    aux_vertex = aux_vertex / np.linalg.norm(aux_vertex)
                    position_candidate = (aux_vertex*parameters["deadlock_mov"]
                                          + agent.pos_i)
            # If deadlock is on an edge
            else:
                #------------------------CONTROL POINT-------------------------
                if verbose:
                    print('Deadlock in edge')
                if sense == 'clockwise':
                    aux_vertex = agent.closer_edge[0] - agent.pos_i
                    aux_vertex = aux_vertex / np.linalg.norm(aux_vertex)
                    position_candidate = (aux_vertex*parameters["deadlock_mov"]
                                          + agent.pos_i)
                elif sense == 'counterclockwise':
                    aux_vertex = agent.closer_edge[1] - agent.pos_i
                    aux_vertex = aux_vertex / np.linalg.norm(aux_vertex)
                    position_candidate = (aux_vertex*parameters["deadlock_mov"]
                                          + agent.pos_i)
            # Make sure the point is inside BVC
            if path.Path(agent.bvc).contains_point(position_candidate):
                agent.closer_point = position_candidate
            else:
                agent.closer_point = agent.pos_i
            if verbose:
                print('-----------------------------------')

### Deadlock Test Function

`def deadlock_test(opt, parameters, robots, step)`

Check if deadlock exits

#### Parameters:
* **opt&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;: dictionary**
    * Dictionary with general options
* **parameters : dictionary**
    * Dictionary with global parameters
* **robots&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: list**
    * List of instances of Robots
* **step&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;: int**
    * Global step of the execution

In [ ]:
def deadlock_test(opt, parameters, robots, step):
    deadlock_flag = False
    if step >= parameters["previous"]:
        for agent in robots:
            if not np.all(agent.closer_point == agent.pos_f):
                j = 0
                agent.deadlock = False
                distance1 = np.linalg.norm(agent.closer_point - agent.pos_i)
                if opt["verbose"]:
                    print('Distance from', agent.color,
                          'to the closest point:', distance1)
                while (not agent.deadlock and j < parameters["previous"]):
                    distance2 = np.linalg.norm(agent.closer_point
                                               - agent.path[-2-j])
                    if opt["verbose"]:
                        print('Distance from the '+str(j+1)+
                              'th previous point:', distance2)
                    if (distance1 < parameters["tolerance"]
                        or distance2 < parameters["tolerance"]):
                        agent.deadlock = True
                        deadlock_flag = True
                    j += 1
    if opt["verbose"]:
        print('Closer positions:')
        for agent in robots:
            print(agent.closer_point)
        print('Deadlock?:', deadlock_flag,'\n')
    # BREAKING DEADLOCK
    if deadlock_flag:
        if opt["verbose"]:
            print('Cells in Deadlock:')
            for agent in robots:
                if agent.deadlock:
                    print(agent.color, end = ' ')
            print('\nCloser_edges:\n')
            for agent in robots:
                if agent.deadlock:
                    print(agent.closer_edge, end = ' ')
        deadlock(robots, parameters, opt["verbose"])
        if opt["verbose"]:
            print('DEADLOCK ABOVE!')
            print('Previous positions:')
            for agent in robots:
                print(agent.path[-2])
            print('New closer positions:\n')
            for agent in robots:
                print(agent.closer_point)